# Numerical project in Python

We will consider the problem of finding the static equilibrium of a chain formed by rigid bars in 2D.
This will be found via an optimization problem with equality constraints.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

## Question 2

Python function that given $z$ as an imput, returns :

$$
c(z) = (c_1 (z) , \ldots , c_{N+1} (z) ) = (l_{i}(x, y)^{2}-L^{2} = \bigl(  (x_{i}-x_{i-1})^{2}+(y_{i}-y_{i-1})^{2}-L^{2} \, , \, i \in [ 1 , \ldots , N+1 ] \bigr)
$$

In [2]:
L=2

In [3]:
def c(z):
    N=np.len(z)/2
    x=z[0:N:1]
    y=z[N:2*N:1]
    c=np.zeros((N+1,1))
    c[0] = (x[0])**2+(y[0])**2 - L**2
    for i in range(1,N)
        l=(x[i]-x[i-1])**2+(y[i]-y[i-1])**2 - L**2
    c[N+1] = (a-x[N-1])**2+(b-y[N-1])**2 - L**2
    return c
    

## Question 3

Python function that given $z$ as an imput, return $\sum_i y_i$.

To do so, we compute a $e = (0 , \ldots , 0 , 1 , \ldots , 1)$ $2n$-array, and we comput the dot product of $e . z$ .

In [4]:
def e(z):
    N=np.len(z)/2
    e=np.zeros(2*N)
    e[N:2*N:1]=np.ones(N)
    return e

In [5]:
def cost(z):
    return np.dot(e(z), z)

## Question 4

Python function that returns the lagrangian of the systeme, given $z$ and $\lambda$ as an imput.

To do so we write the Lagrangian as :

$$
\mathcal{L}(z, \lambda)=e^{\top} z+\lambda^{\top} c(z)
$$

In [6]:
def lag(z,l):
    e=e(z)
    return np.transpose(e)*z + np.transpose(l)*c(z)

## Question 5

Python function, that given $z$, $\lambda$ as an imput, yields $\nabla_z \mathcal{L}(z, \lambda)$.

As shown in the report : 
$$
\nabla_{z} \mathcal{L}(z, \lambda)=e+\nabla_{z}^{\top} c(z) \lambda \in \mathbf{R}^{2 N}
$$

where the Jacobian of c is :
$$
\nabla_{z}^{\top} c(z)=\left[\begin{array}{ccc}
\nabla_{x_{1}} c_{1}(x, y) & \ldots & \nabla_{x_{1}} c_{N+1}(x, y) \\
\nabla_{x_{2}} c_{1}(x, y) & \ldots & \nabla_{x_{2}} c_{N+1}(x, y) \\
\vdots & \vdots & \vdots \\
\nabla_{x_{N}} c_{1}(x, y) & \ldots & \nabla_{x_{N}} c_{N+1}(x, y) \\
\hline \nabla_{y_{1}} c_{1}(x, y) & \ldots & \nabla_{y_{1}} c_{N+1}(x, y) \\
\nabla_{y_{2}} c_{1}(x, y) & \ldots & \nabla_{y_{2}} c_{N+1}(x, y) \\
\vdots & \vdots & \vdots \\
\nabla_{y_{N}} c_{1}(x, y) & \ldots & \nabla_{y_{N}} c_{N+1}(x, y)
\end{array}\right] \in \mathbf{R}^{2 N \times N+1}
$$

First lets compute this Jacobian matrix :

In [10]:
def cjac(z):
    N=np.len(z)/2
    cjac=np.zeros(2*N, N+1)
    cjac[0,0] = 2*z[0]
    cjac[N,0] = 2*z[N]
    cjac[N-1,N] = 2*(z[N-1]-a)
    cjac[2*n-1,N] = 2*(z[N-1]-b)
    for j in range(1,N):
        temp = 2*(z[j]-z[j-1])
        cjac[j,j-1] = -temp
        cjac[j,j] = temp
        temp = 2*(z[N+j]-z[N+j-1])
        cjac[j,N+j-1] = -temp
        cjac[j,N+j] = temp
    return cjac

We deduce the gradient of the Lagrangian by the quick vectorial calculation above

In [13]:
def gradlag(z,l):
    e=e(z)
    cjac = cjac(z)
    return e + np.dot(cjac,l)

This function also check that the derivatives are correct, by checking that :
$$
\frac{\left\|c(z+\delta)-c(z)-\nabla_{z} c(z) \delta\right\|}{\|\delta\|} \leq 0.01
$$

for a small random perturbation $\delta$.

In [15]:
def checkcjac(z,delta):
    cpert = c(z+delta)
    c = c(z)
    cjac_delta = np.dot(cjac(z), delta)
    n1 = np.linlag.norm(cpert - c - cjac_delta)
    n2 = np.linlag.norm(delta)
    if n1/n2 < 0.01:
        return true
    return false